<a href="https://colab.research.google.com/github/JstnClmnt/NLP-Word-Embeddings/blob/master/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd

In [0]:
import os
 
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = dirName+"/"+entry
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles        

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [5]:
!unzip "/content/drive/Shared drives/AI Lords/NLP-Word-Embeddings/Filipino Dataset.zip"

Archive:  /content/drive/Shared drives/AI Lords/NLP-Word-Embeddings/Filipino Dataset.zip
   creating: Filipino Dataset/
  inflating: Filipino Dataset/1.txt  
  inflating: Filipino Dataset/10.txt  
  inflating: Filipino Dataset/100.txt  
  inflating: Filipino Dataset/1001.txt  
  inflating: Filipino Dataset/1002.txt  
  inflating: Filipino Dataset/1003.txt  
  inflating: Filipino Dataset/1004.txt  
  inflating: Filipino Dataset/1005.txt  
  inflating: Filipino Dataset/1006.txt  
  inflating: Filipino Dataset/1007.txt  
  inflating: Filipino Dataset/1008.txt  
  inflating: Filipino Dataset/1009.txt  
  inflating: Filipino Dataset/101.txt  
  inflating: Filipino Dataset/1010.txt  
  inflating: Filipino Dataset/1011.txt  
  inflating: Filipino Dataset/1012.txt  
  inflating: Filipino Dataset/1013.txt  
  inflating: Filipino Dataset/1014.txt  
  inflating: Filipino Dataset/1015.txt  
  inflating: Filipino Dataset/1016.txt  
  inflating: Filipino Dataset/1017.txt  
  inflating: Filipino Data

In [7]:
dirName = 'Filipino Dataset';
    
# Get the list of all files in directory tree at given path
df_predata=pd.read_csv("/content/drive/Shared drives/AI Lords/NLP-Word-Embeddings/filipino.csv")
df_predata=df_predata[["Headline","Authors","File Number","Category"]]
df_predata=df_predata.rename(columns={'File Number':'file_number'}) 
df=pd.DataFrame(columns=["Title","Description","Category"])
for row in df_predata.itertuples():
    file_name=str(row.file_number)+".txt"
    try:
        file1 = open(dirName+"/"+file_name,"r",encoding="utf8") 
    except:
        continue
    #print("Reading "+file_name)
    sampleNews=file1.read().split("\n")
    newsDesc=""
    for strline in sampleNews[1:len(sampleNews)]:
        newsDesc+=strline
    dfsample=pd.DataFrame(columns=["Title","Description","Category"],data=[[row.Headline,newsDesc,row.Category]])
    df=df.append(dfsample)
df.head()

,Title,Description,Category
0,2 nene kinakadena ng magulang sa Albay,"Sa mga larawang ibinigay sa ABS-CBN Bicol, mak...",Crime
0,Hustisya hiling sa pagkamatay ng 'napagkamalan...,Nahuli sa CCTV ang pamamaril ng isang suspek s...,Crime
0,MODUS: Mga batang babae pinaghuhubad kapalit n...,"Noong isang beses, sinabihan siya ng anak na m...",Crime
0,Duterte dumepensa sa ulat ng 'lumalagong yaman',"Sa isang talumpati noong Sabado, sinabi ni Dut...",Politics
0,Mga umuuwi para sa Semana Santa unti-unti nang...,"Sa panayam sa DZMM, sinabi ni Armand Reyes, op...",Vehicles


In [8]:
df=df.reset_index()
df=df.drop(list(df)[0], axis=1)
df["Description"].head()[0]

'Sa mga larawang ibinigay sa ABS-CBN Bicol, makikitang naglalaro ang magkapatid na 11 at 12 anyos pero nakakadena ang mga ito. Ayon sa nagbigay ng retrato, mag-iisang buwan na umano itong ginagawa sa mga bata. Nang puntahan ng Philippine National Police Women and Children\'s Protection Desk (WCPD) ngayong Miyerkoles ang bahay ng mga biktima, umamin umano ang ina na kinakadena ang magkakapatid ng kanilang ama. Katuwiran ng ina na madalas umanong gumagala ang dalawang bata kaya kinakadena sila, pero nakakapaglaro naman sila sa labas ng bahay. Nasasangkot din umano sa pagnanakaw ang magkakapatid. Dagdag pa ng ina na hindi rin daw siya sigurado kung magbabago pa ang kaniyang mga anak.Pero iginiit ng ni Tabaco WCPD chief Police Capt. Maria Theresa Berdin, labag sa batas ang ginawa ng mga magulang. "Very obvious naman na hindi tama na ikadena natin ang ating mga anak dahil lang sa rason na sutil or madalas umalis ng bahay dahil alam naman natin na ang kadena hindi akma sa tao," aniya. Hindi 

In [9]:
df["Category"]=df["Category"].replace("Vehicle", "Vehicles")
df["Category"]=df["Category"].replace("Business ", "Business")
categories=df["Category"].unique().tolist()
cat_dict=dict(zip(categories, range(0,len((categories)))))
print(cat_dict)
for x in categories:
    print(x+": "+str(len(df[df["Category"]==x])))
df["Category"]=df['Category'].map(cat_dict, na_action='ignore')

{'Crime': 0, 'Politics': 1, 'Vehicles': 2, 'Business': 3, 'Entertainment': 4}
Crime: 232
Politics: 250
Vehicles: 207
Business: 248
Entertainment: 250


In [10]:
df['Description'] = df['Description'].str.replace("[^a-zA-Z#]", " ")
df=df.dropna()
df["Description"].head()[0]

'Sa mga larawang ibinigay sa ABS CBN Bicol  makikitang naglalaro ang magkapatid na    at    anyos pero nakakadena ang mga ito  Ayon sa nagbigay ng retrato  mag iisang buwan na umano itong ginagawa sa mga bata  Nang puntahan ng Philippine National Police Women and Children s Protection Desk  WCPD  ngayong Miyerkoles ang bahay ng mga biktima  umamin umano ang ina na kinakadena ang magkakapatid ng kanilang ama  Katuwiran ng ina na madalas umanong gumagala ang dalawang bata kaya kinakadena sila  pero nakakapaglaro naman sila sa labas ng bahay  Nasasangkot din umano sa pagnanakaw ang magkakapatid  Dagdag pa ng ina na hindi rin daw siya sigurado kung magbabago pa ang kaniyang mga anak Pero iginiit ng ni Tabaco WCPD chief Police Capt  Maria Theresa Berdin  labag sa batas ang ginawa ng mga magulang   Very obvious naman na hindi tama na ikadena natin ang ating mga anak dahil lang sa rason na sutil or madalas umalis ng bahay dahil alam naman natin na ang kadena hindi akma sa tao   aniya  Hindi m